## Competative Social Media Analysis

**Explore Data Science Academy** wants to develop a marketing startegy to ensure that all social media content posted reaches the inteded target market or audiance and that our social media market are growing in engagement and also analysing our growing social media engagement as compared to the competitors.

#### Import Standard libraries

In [41]:
import re
import csv
import pandas as pd

import selenium
from selenium import webdriver

from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup as bs

# for visualization
import seaborn as sns
import matplotlib.pyplot as plt

# for extraction
from geograpy import extraction
import nltk
nltk.download('maxent_ne_chunker')

# for organization
from collections import Counter

import time
from datetime import datetime


[nltk_data] Downloading package maxent_ne_chunker to /home/nano-
[nltk_data]     ai/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


#### Configurations

In [42]:
import warnings
warnings.filterwarnings('ignore')

#Disable chrome Notifications
chrome_options = Options()
chrome_options.add_argument("--disable-notifications")

#Install latest chrome drivers
#driver = webdriver.chrome(service=Service(ChromeDriverManager().install()))

#Thanks stackoverflow @pythonjar
#chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

#specify the path to chromedriver.exe (download and save on your computer)
browser = webdriver.Chrome('/home/nano-ai/chromedriver' , chrome_options=chrome_options)

### Automatic Login to Social Media

In [43]:
#Get the Company's Facebook URL from User
page = input("Enter the Company Linkedin URL: ")
company_name = page[33:-1]

#See if existing user credential file exists or create one 
try:
    f= open("linkedin_credentials.txt","r")
    contents = f.read()
    username = contents.replace("=",",").split(",")[1]
    password = contents.replace("=",",").split(",")[3]
except:
    f= open("linkedin_credentials.txt","w+")
    username = input('Enter your linkedin username: ')
    password = input('Enter your linkedin password: ')
    f.write("username={}, password={}".format(username,password))
    f.close()

Enter the Company Linkedin URL: https://www.linkedin.com/company/explore-datascience/


#### Navigate to facebook page/group

In [44]:
#accessing Chromedriver
# browser = webdriver.Chrome('chromedriver')

#Open login page
browser.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')

#Enter login info:
elementID = browser.find_element_by_id('username')
elementID.send_keys(username)

elementID = browser.find_element_by_id('password')
elementID.send_keys(password)
elementID.submit()

In [71]:
#Go to company post webpage
browser.get(page + 'posts/')


#Simulate scrolling to capture all posts
SCROLL_PAUSE_TIME = 1.5

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

### Check page source code

In [72]:
#Check out page source code
company_page = browser.page_source  


#Use Beautiful Soup to get access tags
linkedin_soup = bs(company_page.encode("utf-8"), "html")
linkedin_soup.prettify()

#Find the post blocks
containers = linkedin_soup.findAll("div",{"class":"occludable-update ember-view"})

### Data Scraping

In [73]:
# Lists that we will iterate to
post_dates = []
post_texts = []
post_likes = []
post_comments = []
video_views = []
media_links = []
media_type = []


#Looping through the posts and appending them to the lists
for container in containers:
    
    #Try function to make sure its a post and not a promotion
    try:
        posted_date = container.find("span",{"class":"visually-hidden"})
        text_box = container.find("div",{"class":"feed-shared-text relative feed-shared-update-v2__commentary"})
        text = text_box.find("span",{"dir":"ltr"})
        new_likes = container.findAll("li", {"class":"social-details-social-counts__reactions-count"})
        new_comments = container.findAll("li", {"class": "social-details-social-counts__item social-details-social-counts__comments"})

        #Appending date and text to lists
        post_dates.append(posted_date.get_text())
        post_texts.append(text_box.get_text())


        #Determining media type and collecting relevant info for each type
        try:
            video_box = container.findAll("div",{"class": "feed-shared-linkedin-video__container"})
            video_link = video_box[0].find("video", {"class":"vjs-tech"})
            media_links.append(video_link['src'])
            media_type.append("Video")
        except:
            try:
                image_box = container.findAll("div",{"class": "feed-shared-image__container"})
                image_link = image_box[0].find("img", {"class":"ivm-view-attr__img--centered feed-shared-image__image feed-shared-image__image--constrained lazy-image ember-view"})
                media_links.append(image_link['src'])
                media_type.append("Image")
            except:
                try:
                    image_box = container.findAll("div",{"class": "feed-shared-image__container"})
                    image_link = image_box[0].find("img", {"class":"ivm-view-attr__img--centered feed-shared-image__image lazy-image ember-view"})
                    media_links.append(image_link['src'])
                    media_type.append("Image")
                except:
                    try:
                        article_box = container.findAll("div",{"class": "feed-shared-article__description-container"})
                        article_link = article_box[0].find('a', href=True)
                        media_links.append(article_link['href'])
                        media_type.append("Article")
                    except:
                        try:
                            video_box = container.findAll("div",{"class": "feed-shared-external-video__meta"})          
                            video_link = video_box[0].find('a', href=True)
                            media_links.append(video_link['href'])
                            media_type.append("Youtube Video")   
                        except:
                            try:
                                poll_box = container.findAll("div",{"class": "feed-shared-update-v2__content overflow-hidden feed-shared-poll ember-view"})
                                media_links.append("None")
                                media_type.append("Other: Poll, Shared Post, etc")
                            except:
                                media_links.append("None")
                                media_type.append("Unknown")



        #Getting Video Views. (The folling three lines prevents class name overlap)
        view_container2 = set(container.findAll("li", {'class':["social-details-social-counts__item"]}))
        view_container1 = set(container.findAll("li", {'class':["social-details-social-counts__reactions","social-details-social-counts__comments social-details-social-counts__item"]}))
        result = view_container2 - view_container1

        view_container = []
        for i in result:
            view_container += i

        try:
            video_views.append(view_container[1].text.strip().replace(' Views',''))

        except:
            video_views.append('N/A')

        
        #Appending likes and comments if they exist
        try:
            post_likes.append(new_likes.get_text())
        except:
            post_likes.append(0)
            pass

        try:
            post_comments.append(new_comments[0].text.strip())                           
        except:                                                           
            post_comments.append(0)
            pass
    
    except:
        pass


In [74]:
#Stripping non-numeric values
comment_count = []
for i in post_comments:
    s = str(i).replace('Comment','').replace('s','').replace(' ','')
    comment_count += [s]

In [75]:
attr = {
    "Date Posted": post_dates,
    "Media Type": media_type,
    "Post Text": post_texts,
    "Post Likes": post_likes,
    "Post Comments": comment_count,
    "Video Views": video_views,
    "Media Links": media_links
}

data = pd.DataFrame(attr)

In [79]:
data

,Date Posted,Media Type,Post Text,Post Likes,Post Comments,Video Views,Media Links
0,2 months ago,Image,\n\n\nOne week to go until our part-time 2022 ...,0,0,6 comments,https://media-exp1.licdn.com/dms/image/C5622AQ...
1,3 days ago,Image,"\n\n\nIf data wrangling is your thing, then ma...",0,0,N/A,https://media-exp1.licdn.com/dms/image/C5622AQ...


In [77]:
data['Media Links'][0]

'https://media-exp1.licdn.com/dms/image/C5622AQFBrc4xbNXzmA/feedshare-shrink_800/0/1641802427685?e=2147483647&v=beta&t=09XSfbNCadyU4MIWDlnhcgscoGcRjDsXxkBP6BmEhsM'

### Data Preprocessing

#### Convert Dataframe to CSV File

In [ ]:
#Exporting as csv file to program folder
data.to_csv("{}_posts.csv".format(company_name), encoding='utf-8', index=False)

#### Save csv file to Coud Storage (Aws Elastic File Storage)

### Exploratory Data Analysis (EDA)